***This notebook is intended for the development of a future feature on the website related to a centralized research repository with search.***

## Import Packages

In [1]:
import io
import ijson
import datetime
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
from pymongo import MongoClient
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import word_tokenize
import gensim
from pprint import pprint
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob

In [19]:
import numpy as np

## Load the JSON file

In [2]:

cols = ["reviewerID", "asin", "reviewerName", "helpful_start",
        "helpful_end", "reviewText", "overall", "summary",
        "unixReviewTime", "reviewTime"]

reviewerID = []
asin = []
reviewerName = []
helpful_start = []
helpful_end = []
reviewText = []
overall = []
summary = []
unixReviewTime = []
reviewTime = []
helpful = []

a = datetime.datetime.now()

with open('Health_and_Personal_Care_5.json', encoding="UTF-8") as json_file:
    cursor = 0
    for line_number, line in enumerate(json_file):
        #print ("Processing line", line_number + 1,"at cursor index:", cursor)
        line_as_file = io.StringIO(line)
        # Use a new parser for each line
        json_parser = ijson.parse(line_as_file)
        for prefix, type, value in json_parser:
            if (type == 'string') | (type == 'number'):
                if prefix.strip() == 'reviewerID':
                    reviewerID.append(value)
                    #print(value)
                elif prefix.strip() == 'asin':
                    asin.append(value)
                elif prefix.strip() == 'reviewerName':
                    reviewerName.append(value)
                elif prefix.strip() == 'helpful.item':
                    helpful.append(value)
                elif prefix.strip() == 'reviewText':
                    reviewText.append(value)
                elif prefix.strip() == 'overall':
                    overall.append(value)
                elif prefix.strip() == 'summary':
                    summary.append(value)
                elif prefix.strip() == 'unixReviewTime':
                    unixReviewTime.append(value)
                elif prefix.strip() == 'reviewTime':
                    reviewTime.append(value)
        cursor += len(line)

        
b = datetime.datetime.now()
       
helpful_start = [helpful[i] for i in range(len(helpful)) if i%2 == 0]
helpful_end = [helpful[i] for i in range(len(helpful)) if i%2 == 1]

review = pd.DataFrame(list(zip(reviewerID, asin, reviewerName, helpful_start, helpful_end,
                           reviewText, overall, summary, unixReviewTime, reviewTime)),
               columns =cols)
c=b-a
print(c.seconds)

28


In [3]:
review

,reviewerID,asin,reviewerName,helpful_start,helpful_end,reviewText,overall,summary,unixReviewTime,reviewTime
0,ALC5GH8CAMAI7,159985130X,AnnN,1,1,This is a great little gadget to have around. ...,5.0,Handy little gadget,1294185600,"01 5, 2011"
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""",1,1,I would recommend this for a travel magnifier ...,4.0,Small & may need to encourage battery,1329523200,"02 18, 2012"
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""",75,77,What I liked was the quality of the lens and t...,4.0,Very good but not great,1275955200,"06 8, 2010"
3,A1XZUG7DFXXOS4,159985130X,Cat lover,56,60,Love the Great point light pocket magnifier! ...,4.0,great addition to your purse,1202428800,"02 8, 2008"
4,A1MS3M7M7AM13X,159985130X,Cricketoes,1,1,This is very nice. You pull out on the magnifi...,5.0,Very nice and convenient.,1313452800,"08 16, 2011"
...,...,...,...,...,...,...,...,...,...,...
343299,ABEAUXVVB2E1M,B00J42G2UW,Karinna Ball,1,1,"I am a big fan of electric toothbrushes, and t...",4.0,I love this!,1403136000,"06 19, 2014"
343300,AEC90GPFKLAAW,B00J42G2UW,Laura hogan,1,1,Our kids have used Crest SpinBrushes for sever...,4.0,"Cleans well, but it's not amazing",1401580800,"06 1, 2014"
343301,A1JBUBTQ48ITN4,B00J42G2UW,Onlinesalesgirl,1,1,I have some wisdom teeth and they are awkward ...,5.0,great for cleaning wisdom teeth,1401321600,"05 29, 2014"
343302,ANEEEFP4BL7ZX,B00J42G2UW,"SJ Blouse ""Stacey""",1,1,"Honestly, I almost didn't choose this from Vin...",4.0,Works well and whitens teeth!,1402272000,"06 9, 2014"


In [4]:
# display metadata
review.head()

,reviewerID,asin,reviewerName,helpful_start,helpful_end,reviewText,overall,summary,unixReviewTime,reviewTime
0,ALC5GH8CAMAI7,159985130X,AnnN,1,1,This is a great little gadget to have around. ...,5.0,Handy little gadget,1294185600,"01 5, 2011"
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""",1,1,I would recommend this for a travel magnifier ...,4.0,Small & may need to encourage battery,1329523200,"02 18, 2012"
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""",75,77,What I liked was the quality of the lens and t...,4.0,Very good but not great,1275955200,"06 8, 2010"
3,A1XZUG7DFXXOS4,159985130X,Cat lover,56,60,Love the Great point light pocket magnifier! ...,4.0,great addition to your purse,1202428800,"02 8, 2008"
4,A1MS3M7M7AM13X,159985130X,Cricketoes,1,1,This is very nice. You pull out on the magnifi...,5.0,Very nice and convenient.,1313452800,"08 16, 2011"


In [5]:
review.loc[0]['reviewText']

"This is a great little gadget to have around.  We've already used it to look for splinters and a few other uses.  The light is great.  It's a handy size.  However, I do wish I'd bought one with a little higher magnification."

In [6]:
review.dtypes

reviewerID        object
asin              object
reviewerName      object
helpful_start      int64
helpful_end        int64
reviewText        object
overall           object
summary           object
unixReviewTime     int64
reviewTime        object
dtype: object

In [7]:
# do some data cleaning
full_data = review.copy()
full_data = full_data.drop_duplicates(['reviewerID', 'asin'], keep='first')
full_data = full_data.dropna()

In [8]:
full_data.shape

(343304, 10)

In [9]:
# compute the sentiment rating for reviews
def compute_sentiment_rating(reviews):
    blob = TextBlob(reviews)
    sentiment_rating = blob.sentiment[0]
        
    return sentiment_rating

In [10]:
def get_sentiment(ratings):
    is_positive = []
    if type(ratings) is list: 
        for i in ratings:
            if i > 0:
                is_positive.append(True)
            else:
                is_positive.append(False)
    else: 
        is_positive = True if ratings > 0 else False 

    return is_positive

In [11]:
full_data['sentiment_rating']=full_data['reviewText'].apply(lambda x: compute_sentiment_rating(x))

In [12]:
full_data.head()

,reviewerID,asin,reviewerName,helpful_start,helpful_end,reviewText,overall,summary,unixReviewTime,reviewTime,sentiment_rating
0,ALC5GH8CAMAI7,159985130X,AnnN,1,1,This is a great little gadget to have around. ...,5.0,Handy little gadget,1294185600,"01 5, 2011",0.238889
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""",1,1,I would recommend this for a travel magnifier ...,4.0,Small & may need to encourage battery,1329523200,"02 18, 2012",0.222222
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""",75,77,What I liked was the quality of the lens and t...,4.0,Very good but not great,1275955200,"06 8, 2010",0.320486
3,A1XZUG7DFXXOS4,159985130X,Cat lover,56,60,Love the Great point light pocket magnifier! ...,4.0,great addition to your purse,1202428800,"02 8, 2008",0.301923
4,A1MS3M7M7AM13X,159985130X,Cricketoes,1,1,This is very nice. You pull out on the magnifi...,5.0,Very nice and convenient.,1313452800,"08 16, 2011",0.402667


In [13]:
full_data['is_positive']=full_data['sentiment_rating'].apply(lambda x: True if x > 0 else False)

In [14]:
full_data.head()

,reviewerID,asin,reviewerName,helpful_start,helpful_end,reviewText,overall,summary,unixReviewTime,reviewTime,sentiment_rating,is_positive
0,ALC5GH8CAMAI7,159985130X,AnnN,1,1,This is a great little gadget to have around. ...,5.0,Handy little gadget,1294185600,"01 5, 2011",0.238889,True
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""",1,1,I would recommend this for a travel magnifier ...,4.0,Small & may need to encourage battery,1329523200,"02 18, 2012",0.222222,True
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""",75,77,What I liked was the quality of the lens and t...,4.0,Very good but not great,1275955200,"06 8, 2010",0.320486,True
3,A1XZUG7DFXXOS4,159985130X,Cat lover,56,60,Love the Great point light pocket magnifier! ...,4.0,great addition to your purse,1202428800,"02 8, 2008",0.301923,True
4,A1MS3M7M7AM13X,159985130X,Cricketoes,1,1,This is very nice. You pull out on the magnifi...,5.0,Very nice and convenient.,1313452800,"08 16, 2011",0.402667,True


In [16]:
# get the first 5000 articles for development
subset = full_data.head(5000)

# convert to lower case
subset['lower_reviewText'] = subset['reviewText'].str.lower()
# tokenize body of research articles
subset['lower_reviewText_tokens'] = subset['lower_reviewText'].apply(lambda x: word_tokenize(str(x)))
# remove non alphanumeric characters
subset['lower_reviewText_tokens'] = subset['lower_reviewText_tokens'].apply(lambda x: [w for w in x if w.isalpha()])

# convert to lower case
subset['lower_summary'] = subset['summary'].str.lower()
# tokenize body of research articles
subset['lower_summary_tokens'] = subset['lower_summary'].apply(lambda x: word_tokenize(str(x)))
# remove non alphanumeric characters
subset['lower_summary_tokens'] = subset['lower_summary_tokens'].apply(lambda x: [w for w in x if w.isalpha()])

C:\Users\mites\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\mites\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\mites\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [17]:
subset.head()

,reviewerID,asin,reviewerName,helpful_start,helpful_end,reviewText,overall,summary,unixReviewTime,reviewTime,sentiment_rating,is_positive,lower_reviewText,lower_reviewText_tokens,lower_summary,lower_summary_tokens
0,ALC5GH8CAMAI7,159985130X,AnnN,1,1,This is a great little gadget to have around. ...,5.0,Handy little gadget,1294185600,"01 5, 2011",0.238889,True,this is a great little gadget to have around. ...,"[this, is, a, great, little, gadget, to, have,...",handy little gadget,"[handy, little, gadget]"
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""",1,1,I would recommend this for a travel magnifier ...,4.0,Small & may need to encourage battery,1329523200,"02 18, 2012",0.222222,True,i would recommend this for a travel magnifier ...,"[i, would, recommend, this, for, a, travel, ma...",small & may need to encourage battery,"[small, may, need, to, encourage, battery]"
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""",75,77,What I liked was the quality of the lens and t...,4.0,Very good but not great,1275955200,"06 8, 2010",0.320486,True,what i liked was the quality of the lens and t...,"[what, i, liked, was, the, quality, of, the, l...",very good but not great,"[very, good, but, not, great]"
3,A1XZUG7DFXXOS4,159985130X,Cat lover,56,60,Love the Great point light pocket magnifier! ...,4.0,great addition to your purse,1202428800,"02 8, 2008",0.301923,True,love the great point light pocket magnifier! ...,"[love, the, great, point, light, pocket, magni...",great addition to your purse,"[great, addition, to, your, purse]"
4,A1MS3M7M7AM13X,159985130X,Cricketoes,1,1,This is very nice. You pull out on the magnifi...,5.0,Very nice and convenient.,1313452800,"08 16, 2011",0.402667,True,this is very nice. you pull out on the magnifi...,"[this, is, very, nice, you, pull, out, on, the...",very nice and convenient.,"[very, nice, and, convenient]"


In [22]:
cluster_std = np.std(subset['sentiment_rating'].tolist())

In [24]:
review_string = 'I hate this product it\'s so bad'
sentiment_rating = compute_sentiment_rating(review_string)
if sentiment_rating < 0:
    lower_limit = (abs(sentiment_rating) - cluster_std) if (abs(sentiment_rating) - cluster_std) > 0 else 0
    upper_limit = abs(sentiment_rating) + cluster_std
else:
    lower_limit = sentiment_rating
    upper_limit = sentiment_rating + (1.5 * cluster_std)

In [25]:
sentiment_rating

-0.75

In [26]:
lower_limit

0.531604680949491

In [27]:
upper_limit

0.968395319050509

In [15]:
stop na

SyntaxError: invalid syntax (<ipython-input-15-d5bced4d2fd1>, line 1)

In [ ]:
stop_words = stopwords.words('english')
def remove_stopwords(word_tokens):
    ''' helped function to remove stopwords '''
    return [w for w in word_tokens if not w in stop_words]

In [ ]:
subset['lower_reviewText_tokens'] = subset['lower_reviewText_tokens'].apply(lambda x: remove_stopwords(x))
subset['lower_summary_tokens'] = subset['lower_summary_tokens'].apply(lambda x: remove_stopwords(x))

In [ ]:
all_reviewText = list(subset['lower_reviewText_tokens'])
model = gensim.models.Word2Vec(all_reviewText, size=100, window=5, min_count=1)
word_vectors = model.wv

In [ ]:
all_summary = list(subset['lower_summary_tokens'])
model = gensim.models.Word2Vec(all_summary, size=100, window=5, min_count=1)
word_vectors = model.wv

In [ ]:
!pip install textblob
from textblob import TextBlob

In [ ]:
all_reviewText[0]

In [ ]:
all_summary[0]

In [ ]:
blob = TextBlob(review.loc[0]['reviewText'])
sentiment_rating = blob.sentiment[0]

In [ ]:
sentiment_rating

In [ ]:
model.most_similar(positive='coronavirus')

In [ ]:
model.most_similar(positive='transmission')

In [ ]:
# define a search string
search_string = 'What do we know about the transmission of the virus?'

In [ ]:
# get distances of each paper against the search string
distances = [(i[1]['paper_id'], 
              model.wmdistance(search_string, 
                               i[1]['body'])) for i in full_data.iterrows()]

In [ ]:
# get the first 5 elements
sorted(distances,key=lambda x: x[1])[:5]

In [ ]:
# sample and see research title
full_data.loc[full_data['paper_id'] == 'PMC7121202']['title'].iloc[0]

In [ ]:
# randomly sample and see research text
full_data.loc[full_data['paper_id'] == 'PMC7121202']['new_abstract'].iloc[0]

In [ ]:
# perform LDA
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(subset['body'].values.astype('U'))

In [ ]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.partial_fit(doc_term_matrix)

In [ ]:
doc_term_matrix

In [ ]:
LDA

In [ ]:
first_topic =LDA.components_[0]

In [ ]:
for i in first_topic.argsort()[-20:]:
    print(count_vect.get_feature_names()[i])

In [ ]:
count_vect.get_feature_names()[i]

In [ ]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

In [ ]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

In [ ]:
subset['Topic'] = topic_values.argmax(axis=1)

In [ ]:
subset['Topic'].unique()

In [ ]:
subset.head()